In [1]:
import pandas as pd
import numpy as np
import os


# DTM2020

## Loading Functions

In [2]:


def load__inputs(path_msisin):
    msisinputs = pd.read_csv(path_msisin, 
                        dtype=object,
                         skiprows=1,
                        names = ['Year',
                                 'Month',
                                 'Day',
                                 'Hour(UTC)',
                                 'Minute(UTC)',
                                 'Second(UTC)',
                                 'Latitude(deg)',
                                 'Longitude(deg)',
                                 'Altitude(km)',
                                 'Cd',
                                 'Aref (m2)',
                                     ],
                        sep = ',',
                        )
    dfdates = pd.DataFrame({'year'   : msisinputs['Year'],
                        'month'  : msisinputs['Month'],
                        'day'    : msisinputs['Day'],
                        'hour'   : msisinputs['Hour(UTC)'],
                        'minute' : msisinputs['Minute(UTC)'],
                        'second' : msisinputs['Second(UTC)'],
                       })
#     msisinputs['Dates'] =  pd.to_datetime(dfdates)
    msisinputs.insert(0, 'Dates', pd.to_datetime(dfdates))

    #### Delete the superfluous columns
    del msisinputs['Year']
    del msisinputs['Month']
    del msisinputs['Day']
    del msisinputs['Hour(UTC)']
    del msisinputs['Minute(UTC)']
    del msisinputs['Second(UTC)']
    return(msisinputs)


##### FUNCTION TO READ THE netcdf F10.7 file
from netCDF4 import Dataset
def read_nc_file( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue.  '''
    status = os.path.exists(filename)
    
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )



### Load Flux and Geomag

In [3]:
# print(ncdump(path_to_mgii))
data_path = '/data/data_geodyn/'
path_to_f107 = data_path + 'gpi_1960001-2022040.nc'
variables = ['year_day', 'f107d', 'f107a', 'kp']
# mgII_data = read_nc_file(path_to_mgii, variables)


arc_list = []
arc_list_2022 = np.arange(32,40)
for i in arc_list_2022:
    val = '2022'+'0'+str(i)
    arc_list.append(int(val))
print('Will load GPI data for the following days: \n',arc_list)

Will load GPI data for the following days: 
 [2022032, 2022033, 2022034, 2022035, 2022036, 2022037, 2022038, 2022039]


In [4]:
#################################################################
##########    First fix the F10.7 values to be MgII    ##########
#################################################################
Kp_to_Ap = {0.    : 0  ,
            0.1   : 0  ,
            0.2   : 1  ,
            0.3   : 2  ,
            0.7   : 3  ,
             1.   : 4  ,
             1.3  : 5  ,
             1.7  : 6  ,
             2.   : 7  ,
             2.3  : 9  ,
             2.7  : 12 ,
             3.   : 15 ,
             3.3  : 18 ,
             3.7  : 22 ,
             4.   : 27 ,
             4.3  : 32 ,
             4.7  : 39 ,
             5    : 48 ,
             5.3  : 56 ,
             5.7  : 67 ,
             6.   : 80 ,
             6.3  : 94 ,
             6.7  : 111,
             7    : 132,
             7.3  : 154,
             7.7  : 179,
             8.   : 207,
             8.3  : 236,
             8.7  : 300,
             9.   : 400,
           }

Ap_to_Kp = {0   :0.   ,
            0   :0.1  ,
            1   :0.2  ,
            2   :0.3  ,
            3   :0.7  ,
            4   : 1.  ,
            5   : 1.3 ,
            6   : 1.7 ,
            7   : 2.  ,
            9   : 2.3 ,
            12  : 2.7 ,
            15  : 3.  ,
            18  : 3.3 ,
            22  : 3.7 ,
            27  : 4.  ,
            32  : 4.3 ,
            39  : 4.7 ,
            48  : 5   ,
            56  : 5.3 ,
            67  : 5.7 ,
            80  : 6.  ,
            94  : 6.3 ,
            111 : 6.7 ,
            132 : 7   ,
            154 : 7.3 ,
            179 : 7.7 ,
            207 : 8.  ,
            236 : 8.3 ,
            300 : 8.7 ,
            400 : 9.  ,
           }


variables    =  ['year_day', 'f107d', 'f107a', 'kp']
f107_data = read_nc_file(path_to_f107, variables)
f107_data    =  f107_data[0]
f107_data['Date']= pd.to_datetime( f107_data['year_day'], format='%Y%j')

#### Make an array of datetimes for the 3hr Kps
doy=[]
Date_3hrAp = []
Ap = []
Kp = []
Ap_dailyavg = []

for i,val in enumerate(f107_data['Date']):
    doy.append(str(f107_data['year_day'][i])[-3:])

    n=0
    avg_ap = []
    for ii,kpval in enumerate(f107_data['kp'][i]):
        Ap.append( Kp_to_Ap[np.around(kpval,1)])
        Kp.append( np.around(kpval,1))
        avg_ap.append( Kp_to_Ap[np.around(kpval,1)])

        if ii==0:
            Date_3hrAp.append(val)
        else:
            Date_3hrAp.append(val + pd.Timedelta(hours=3*n))
        n+=1

    Ap_dailyavg.append(np.mean(avg_ap))
f107_data['Date_3hrAp']=  pd.to_datetime(Date_3hrAp)
f107_data['DOY']= doy
f107_data['Ap']= Ap
f107_data['Kp']= Kp
f107_data['Ap_dailyavg']= Ap_dailyavg


f107d_earth = []
f107a_earth = []

for i,val in enumerate(f107_data['Date']):
    iday = int(f107_data['DOY'][i])

    ## Account for the F10.7 at earth (instead of referenced at 1AU)
    theta0 = 2 * np.pi * (iday)/365.
    sfeps = 1.000110 + 0.034221*np.cos(theta0)+0.001280* np.sin(theta0) +0.000719*np.cos(2.*theta0)+0.000077*np.sin(2.*theta0)

    f107d_earth.append(sfeps * f107_data['f107d'][i])
    f107a_earth.append(sfeps * f107_data['f107a'][i])

f107_data['f107d_earth'] = f107d_earth
f107_data['f107a_earth'] = f107a_earth

#################################################################
            #  C    AP - MAGNETIC INDEX(DAILY) OR WHEN SW(9)=-1. :
        #  C      - ARRAY CONTAINING:
        #  C       (1) DAILY AP
        #  C       (2) 3 HR AP INDEX FOR CURRENT TIME
        #  C       (3) 3 HR AP INDEX FOR 3 HRS BEFORE CURRENT TIME
        #  C       (4) 3 HR AP INDEX FOR 6 HRS BEFORE CURRENT TIME
        #  C       (5) 3 HR AP INDEX FOR 9 HRS BEFORE CURRENT TIME
        #  C       (6) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 12 TO 33 HRS PRIOR
        #  C          TO CURRENT TIME
        #  C       (7) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 36 TO 57 HRS PRIOR
        #  C          TO CURRENT TIME
#################################################################



    
    
# f107_data = read_nc_file(path_to_f107, variables)

# date = []
# kp_list = []
# f107d_list = []
# f107a_list  = []
# date_3hr = []
# doy_list    = []



# for i,val in enumerate(arc_list):
    
#     index = f107_data[0]['year_day']==val
#     kp_list.append(f107_data[0]['kp'][index][0])
#     f107d_list.append(f107_data[0]['f107d'][index][0])
#     f107a_list.append(f107_data[0]['f107a'][index][0])
#     doy_list.append(str(f107_data[0]['year_day'][index][0])[-3:])

#     date.append(pd.to_datetime( str(val), format='%Y%j'))

#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=0))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=3))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=6))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=9))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=12))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=15))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=18))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=21))
# #     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=24))
    
# kp_expand = []
# for i in kp_list:
#     for ii in i:
#         kp_expand.append(ii)
        
        
        
# solar_fluxes = {}
# geomag_fluxes = {}
# solar_fluxes['f107d_list'] = f107d_list
# solar_fluxes['f107a_list'] = f107a_list
# solar_fluxes['date']       = date
# geomag_fluxes['date_3hr']   = date_3hr
# geomag_fluxes['kp_expand']  = kp_expand

# f107d_earth = []
# f107a_earth = []
# ######################################################################### 
# ##### Account for the F10.7 at earth (instead of referenced at 1AU) #####
# ######################################################################### 

# for i_doy,val_doy in enumerate(doy_list):
#     iday = int(val_doy)
#     theta0 = 2 * np.pi * (iday)/365.
#     sfeps = 1.000110 + 0.034221*np.cos(theta0)+0.001280* np.sin(theta0) +0.000719*np.cos(2.*theta0)+0.000077*np.sin(2.*theta0)

#     f107d_earth.append(sfeps * solar_fluxes['f107d_list'][i_doy])
#     f107a_earth.append(sfeps * solar_fluxes['f107a_list'][i_doy])

# solar_fluxes['f107d_earth'] = f107d_earth
# solar_fluxes['f107a_earth'] = f107a_earth

# solar_fluxes_df = 0
# geomag_fluxes_df = 0
# solar_fluxes_df = pd.DataFrame.from_dict(solar_fluxes)
# geomag_fluxes_df = pd.DataFrame.from_dict(geomag_fluxes)


In [5]:
#### clear up some space        
truncate_date    = np.logical_and(f107_data['Date'].year>=2022 , f107_data['Date'].year<=2023 )
truncate_date3hr =  np.logical_and(f107_data['Date_3hrAp'].year>=2022 , f107_data['Date_3hrAp'].year<=2023 )#np.logical_and(mgII_data['Date_3hrAp'].year==year , mgII_data['Date_3hrAp'].dayofyear==day )

f107_data['Date_3hrAp'] = f107_data['Date_3hrAp'][truncate_date3hr]
f107_data['Ap']         = np.array(f107_data['Ap'])[truncate_date3hr]
f107_data['Date']       = f107_data['Date'][truncate_date]
f107_data['Ap_dailyavg'] =  np.array(f107_data['Ap_dailyavg'])[truncate_date]
f107_data['f107d_earth'] = np.array(f107_data['f107d_earth'])[truncate_date]
f107_data['f107a_earth'] = np.array(f107_data['f107a_earth'])[truncate_date]

del f107_data['DOY']
del f107_data['kp']
del f107_data['f107d']
del f107_data['f107a']
del f107_data['year_day']


In [6]:
# %load_ext autoreload
# %autoreload 2
# from numpy import array
# import sys

# #### MAKE MSIS Take the 3HR Ap values
# from pymsis import msis
# SWI_option = [1.0]*25
# SWI_option[8] = -1.0
#             #  C    AP - MAGNETIC INDEX(DAILY) OR WHEN SW(9)=-1. :
#             #  C      - ARRAY CONTAINING:
#             #  C       (1) DAILY AP
#             #  C       (2) 3 HR AP INDEX FOR CURRENT TIME
#             #  C       (3) 3 HR AP INDEX FOR 3 HRS BEFORE CURRENT TIME
#             #  C       (4) 3 HR AP INDEX FOR 6 HRS BEFORE CURRENT TIME
#             #  C       (5) 3 HR AP INDEX FOR 9 HRS BEFORE CURRENT TIME
#             #  C       (6) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 12 TO 33 HRS PRIOR
#             #  C          TO CURRENT TIME
#             #  C       (7) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 36 TO 57 HRS PRIOR
#             #  C          TO CURRENT TIME


# # ### DATE HANDLING
# # def nearest(items, pivot):
# #     return min(items, key=lambda x: abs(x - pivot))


# # vals_flux  = np.arange(solar_fluxes_df.index[0],solar_fluxes_df.index[-1]+1)
# # df_flux = solar_fluxes_df.set_index('date',drop=False) 
# # df_flux['i_vals'] = vals_flux

# # vals_geo  = np.arange(geomag_fluxes_df.index[0],geomag_fluxes_df.index[-1]+1)
# # df_geomag = geomag_fluxes_df.set_index('date_3hr',drop=False) 
# # df_geomag['i_vals'] = vals_geo


# msisinputs['Rho(kg/m3)'] = np.zeros(np.size(msisinputs['Dates']))
# msisinputs['f107d(ref@1AU)'] = np.zeros(np.size(msisinputs['Dates']))
# msisinputs['f107a(ref@1AU)'] = np.zeros(np.size(msisinputs['Dates']))
# msisinputs['Ap_daily_avg'] = np.zeros(np.size(msisinputs['Dates']))
# msisinputs['Ap_3HR_current'] = np.zeros(np.size(msisinputs['Dates']))
# msisinputs['Ap_3HR_prior'] = np.zeros(np.size(msisinputs['Dates']))
# msisinputs['Ap_6HR_prior'] = np.zeros(np.size(msisinputs['Dates']))
# msisinputs['Ap_9HR_prior'] = np.zeros(np.size(msisinputs['Dates']))
# msisinputs['Ap_12hr_33hr_priorAVG'] = np.zeros(np.size(msisinputs['Dates']))
# msisinputs['Ap_36hr_57hr_priorAVG'] = np.zeros(np.size(msisinputs['Dates']))

# for i,dateval in enumerate(msisinputs['Dates']):
    
#     dates  = dateval
# #     f107  = solar_fluxes_df['f107d_earth'][index_fluxdate]
# #     f107a = solar_fluxes_df['f107a_earth'][index_fluxdate]

# #     dateflux_near = nearest(pd.to_datetime(df_flux['date'].index), dateval)
# #     index_fluxdate = df_flux['i_vals'][dateflux_near]   
# #     dategeomag_near = nearest(pd.to_datetime(df_geomag['date_3hr'].index), dateval)
# #     index_geodate = df_geomag['i_vals'][dategeomag_near]
    
#     ####---------------------------------------------
#     #### Gather the Necessary Flux and Ap Information
#     index_date = np.logical_and(f107_data['Date'].year==dateval.year , f107_data['Date'].dayofyear==dateval.day )
#     f107a = float(np.squeeze(np.asarray(f107_data['f107a_earth'])[index_date]))
#     f107d = float(np.squeeze(np.asarray(f107_data['f107d_earth'])[index_date]))
#     Ap_daily_avg = float(np.squeeze(np.asarray(f107_data['Ap_dailyavg'])[index_date]))
#     f107din = [f107d]
#     f107ain = [f107a]

#     ##### ---- CONSTRUCT STORMTIME AP VALUES
#     #####----------------------------------------------------------------
    

# #     date.append(dateval) 

#     index_date3hr = np.logical_and(f107_data['Date_3hrAp'].year==dateval.year , f107_data['Date_3hrAp'].dayofyear==dateval.day  )
#     indexvals =  [i for i, x in enumerate(index_date3hr) if x]
#     Ap_doy_windows = f107_data['Date_3hrAp'][indexvals]

#     #### Find the Current 3hr Kp window:A
#     Ap_windw_hrs = [i.hour for i in Ap_doy_windows]
#     Ap_windw_hrs = np.append(np.array(Ap_windw_hrs),24)  ## add the final window edge

#     index_current_Ap = int(np.digitize([dates.hour],Ap_windw_hrs))
#     if index_current_Ap==8:
#         index_current_Ap += -1
#     indexglobal_currentAp = indexvals[index_current_Ap]

#     Ap_3HR_current        = f107_data['Ap'][indexglobal_currentAp]
#     Ap_3HR_prior          = f107_data['Ap'][indexglobal_currentAp-1]
#     Ap_6HR_prior          = f107_data['Ap'][indexglobal_currentAp-2]
#     Ap_9HR_prior          = f107_data['Ap'][indexglobal_currentAp-3]
#     Ap_12hr_33hr_priorAVG = np.mean(f107_data['Ap'][indexglobal_currentAp-11 :indexglobal_currentAp-3 ] ) ### 33hrs to 12 hours
#     Ap_36hr_57hr_priorAVG = np.mean(f107_data['Ap'][indexglobal_currentAp-19 :indexglobal_currentAp-11 ])  ### 36hrs to 57 hours

#     apsin = [[Ap_daily_avg,          # (1) DAILY AP
#               Ap_3HR_current,        # (2) 3 HR AP INDEX FOR CURRENT TIME
#               Ap_3HR_prior,          # (3) 3 HR AP INDEX FOR 3 HRS BEFORE CURRENT TIME
#               Ap_6HR_prior,          # (4) 3 HR AP INDEX FOR 6 HRS BEFORE CURRENT TIME
#               Ap_9HR_prior,          # (5) 3 HR AP INDEX FOR 9 HRS BEFORE CURRENT TIME
#               Ap_12hr_33hr_priorAVG, # (6) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 12 TO 33 HRS PRIOR
#               Ap_36hr_57hr_priorAVG]]# (7) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 36 TO 57 HRS PRIOR

#     #####----------------------------------------------------------------
    
#     lat  = float(msisinputs['Latitude(deg)'][i])
#     lon  = float(msisinputs['Longitude(deg)'][i])
#     alt  = float(msisinputs['Altitude(km)'][i])

#     output2 = msis.run(dateval, lon, lat, alt, f107din, f107ain, apsin, version = 2, options=SWI_option)
# #     output2 = msis.run(date, lon, lat, alt, f107, f107a, aps, version = 2)
    
#     msisinputs.loc[i, ['Rho(kg/m3)']]     = output2[0][0][0][0][0]
#     msisinputs.loc[i, ['Ap_daily_avg']]   = Ap_daily_avg
#     msisinputs.loc[i, ['Ap_3HR_current']] = Ap_3HR_current
#     msisinputs.loc[i, ['Ap_3HR_prior']]   = Ap_3HR_prior
#     msisinputs.loc[i, ['Ap_6HR_prior']]   = Ap_6HR_prior
#     msisinputs.loc[i, ['Ap_9HR_prior']]   = Ap_9HR_prior
#     msisinputs.loc[i, ['Ap_12hr_33hr_priorAVG']] = Ap_12hr_33hr_priorAVG
#     msisinputs.loc[i, ['Ap_36hr_57hr_priorAVG']] = Ap_36hr_57hr_priorAVG
#     msisinputs.loc[i, ['f107d(ref@1AU)']]  = f107din
#     msisinputs.loc[i, ['f107a(ref@1AU)']] = f107ain

    
# print(msisinputs)

### Plot to Check

In [7]:
# import plotly.graph_objects as go
# from plotly.offline import plot, iplot
# from plotly.subplots import make_subplots
# import plotly.express as px
# import plotly.io as pio   ### Allows you to save plotly figs


# import pandas as pd
# import datetime
# import numpy as np

# config = dict({
#                 'displayModeBar': False,
#                 'responsive': False,
#                 'staticPlot': False,
#                 'displaylogo': False,
#                 'showTips': False,
#                 })


In [8]:
# fig = make_subplots(rows=3, cols=1,
# #                     vertical_spacing = 0.03,
#                     shared_xaxes=True)



# fig.add_trace(go.Scattergl(y=msisinputs['Rho(kg/m3)'],
#                            x=msisinputs['Dates'],
#                            name= 'MSIS2.0 Rho',
#                            mode='markers',
#                            opacity=1,
# #                            line = dict(shape = 'hvh',dash='dash', color = 'black', width=2),
#                              marker=dict(size=4),
#                          showlegend=False),
#                            secondary_y=False,row=1, col=1)
# fig.add_trace(go.Scattergl(y=msisinputs['f107d(ref@1AU)'],
#                            x=msisinputs['Dates'],
#                            name= 'f107d',
#                            mode='lines',
#                            opacity=1,
#                            line = dict(shape = 'hvh',dash='dash', color = 'grey', width=2),
#                              marker=dict(size=4),
#                          showlegend=False),
#                            secondary_y=False,row=2, col=1)
# fig.add_trace(go.Scattergl(y=msisinputs['f107a(ref@1AU)'],
#                            x=msisinputs['Dates'],
#                            name= 'f107a',
#                            mode='lines',
#                            opacity=1,
#                            line = dict(shape = 'hvh',dash='solid', color = 'black', width=2),
#                              marker=dict(size=4),
#                          showlegend=False),
#                            secondary_y=False,row=2, col=1)
# fig.add_trace(go.Scattergl(y=msisinputs['Ap_3HR_current'],
#                            x=msisinputs['Dates'],
#                            name= 'Ap_3hr',
#                            mode='lines',
#                            opacity=1,
#                            line = dict(shape = 'hvh',dash='solid', color = 'black', width=2),
#                              marker=dict(size=4),
#                          showlegend=False),
#                            secondary_y=False,row=3, col=1)

# #######################################################
# font_dict=dict(family='Arial',size=16,color='black')
# #######################################################

# fig.update_yaxes(title_text="Rho (kg/m3)", 
#                  type="log", 
#                  exponentformat= 'power',row=1, col=1)
# fig.update_yaxes(title_text="F10.7", 
#                  exponentformat= 'power',row=2, col=1)
# fig.update_yaxes(title_text="Ap", 
#                  exponentformat= 'power',row=3, col=1)

   
# for i in [1,2, 3]:
#     if i == 3:
#         xlabel=True
#     else:
#         xlabel=False
    
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,showticklabels=xlabel,
#                       linecolor='black',linewidth=1,
#                      ### Major ticks
#                       ticks='inside',tickfont=font_dict,mirror=True,
#                       tickwidth=2,ticklen=9,
#                       tickcolor='grey',tick0="2018-11-9" ,
#                       dtick=86400000.0*2,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,ticklen=4,
#                          tickcolor='grey',ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',gridwidth=1,
#                        layer='above traces',tickangle=-45,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                      showticklabels=True,linecolor='black',  
#                      linewidth=1,ticks='inside',tickfont=font_dict, 
#                      mirror='allticks',tickwidth=1,tickcolor='black',
#                      gridcolor='gainsboro',gridwidth=1,layer='above traces',
#                      row=i, col=1)


# fig.update_layout(autosize=False,    width=900,    height=800,
#               legend= {'itemsizing': 'trace'},
#               font=font_dict, plot_bgcolor='white', 
#              )
# fig.update_annotations(font_size=14)  # Increase size of subplot title
# fig.show(#renderer="jpg",
#      config=dict({
#                 'displayModeBar': False,
#                 'responsive': False,
#                 'staticPlot': False,
#                 'displaylogo': False,
#                 'showTips': False,
#                 }))



In [9]:
# msisinputs.columns

In [10]:
# path_msisout = "/data/zach_work/side_projects/ManualModelRuns/starlink3167_ephemeris_full_MSIS2p0.csv"

# msisinputs.to_csv(path_msisout,
#                   sep=',',
#                   columns=msisinputs.columns, 
#                   header=True)

In [11]:
# import pandas as pd
# check = pd.read_csv(path_msisout, 
#                          dtype=object,
#                          skiprows=1,
#                          names = ['index'
#                                   'Dates',
#                                   'Latitude(deg)',
#                                   'Longitude(deg)',
#                                   'Altitude(km)',
#                                   'Cd', 
#                                   'Aref (m2)',
#                                   'Rho(kg/m3)',
#                                   'f107d(ref@1AU)',
#                                   'f107a(ref@1AU)',
#                                   'Ap_daily_avg',
#                                   'Ap_3HR_current',
#                                   'Ap_3HR_prior',
#                                   'Ap_6HR_prior',
#                                   'Ap_9HR_prior',
#                                   'Ap_12hr_33hr_priorAVG',
#                                   'Ap_36hr_57hr_priorAVG'],
#                     sep = ',',
#                     )


```fortran
!               from dtm2020 source code:
                  ! ** INPUTS **
                  !     day   =   day of year [1-366]
                  !     f     =   f(1) = instantaneous flux at (t - 24hr)    
                  !               f(2) = 0.
                  !     fbar  =   fbar(1) = mean flux of last 81 days at t  
                  !               fbar(2) = 0.
                  !     akp   =   akp(1) = kp delayed by 3 hours, 
                  !               akp(3) = mean of last 24 hours,
                  !               akp(2) = 0. 
                  !               akp(4) = 0.
                  !     alti  =   altitude (in km) greater than 120 km
                  !     hl    =   local time (in radian: 0-24hr = 0-2pi)
                  !     alat  =   latitude (in radian)
                  !     xlon  =   longitude (in radian)
                  !
                  ! ** OUTPUT **
                  !     tz     =  temperature at altitude -> alti
                  !     tinf   =  exospheric temperature
                  !     d(1)   =  partial density of atomic hydrogen (in gram/cm3)
                  !     d(2)   =  partial density of helium
                  !     d(3)   =  partial density of atomic oxygen
                  !     d(4)   =  partial density of molecular nitrogen
                  !     d(5)   =  partial density of molecular oxygen
                  !     d(6)   =  partial density of atomic nitrogen
                  !     ro     =  total density (in gram/cm3)
                  !     wmm    =  mean molecular mass (in gram)


      call dtm3(DOY_real,                           &
          &     flux_daily,flux_avg,kp_array,      &
          &     ALTKM,STLOC,GLAT,GLON,             &
          &     Temp_z,Temp_exo,                   &
          &     RHO_dtm2020,part_dens,           &
          &     MBAR_dtm2020)
```

# Prep Variables for DTM

In [12]:
path_datain = "/data/zach_work/side_projects/ManualModelRuns/starlink3167_ephemeris_full.csv"
data_pre = load__inputs(path_datain)

In [13]:
data_pre['Dates'][0].day

3

In [14]:
filler =  np.zeros(np.size(data_pre['Dates'])) 
datafill = {'DOY'  : filler,
         'f1'  : filler,
         'f2'  : filler,
         'fbar1'  : filler,
         'fbar2'  : filler,
         'kp1'  : filler,
         'kp2'  : filler,
         'kp3'  : filler,
         'kp4'  : filler,
         'alt_km'  : filler,
         'localtime_rads'  : filler,
         'lat_rads'  : filler,
         'lon_rads'  : filler,
       }
data_DTM_in = pd.DataFrame(data=datafill)



In [15]:
from numpy import array
import numpy as np
import sys


for i,dateval in enumerate(data_pre['Dates']):
    # DOY              # day of year [1-366]
    # f1               # instantaneous flux at (t - 24hr)  
    # f2               = 0
    # fbar1            # mean flux of last 81 days at t 
    # fbar2            = 0
    # kp1              # kp delayed by 3 hours
    # kp2              # mean of last 24 hours
    # kp3              = 0
    # kp4              = 0
    # alt_km           # altitude (in km) greater than 120 km
    # localtime_rads   # local time (in radian: 0-24hr = 0-2pi)
    # lat              # latitude (in radian)
    # lon              # longitude (in radian)

    data_DTM_in.loc[i, ['DOY']]     = data_pre['Dates'][0].day

#     1deg * pi/180 = 0.01745rad
    data_DTM_in.loc[i, ['lat_rads']] = float(data_pre['Latitude(deg)'][i]) * np.pi/180
    data_DTM_in.loc[i, ['lon_rads']] = float(data_pre['Longitude(deg)'][i])* np.pi/180
    data_DTM_in.loc[i, ['alt_km']]   = float(data_pre['Altitude(km)'][i])

    
    
    ut = float(str(data_pre['Dates'][i].hour) +'.'+ str(data_pre['Dates'][i].minute))
    calc_slt_hrs = np.mod(float(data_pre['Longitude(deg)'][i])/15 + ut, 24)
    slt_rads   =calc_slt_hrs*15 *np.pi/180
#     print(calc_slt_hrs)
#     print(slt_rads)
#     print()
    data_DTM_in.loc[i, ['localtime_rads']]   = slt_rads 

    
    data_DTM_in.loc[i, ['f2']]    = 0
    data_DTM_in.loc[i, ['fbar2']] = 0 
    data_DTM_in.loc[i, ['kp3']]   = 0
    data_DTM_in.loc[i, ['kp4']]   = 0

    
    
    ####---------------------------------------------
    #### Gather the Necessary Flux and Ap Information
    index_date = np.logical_and(f107_data['Date'].year==dateval.year , f107_data['Date'].dayofyear==dateval.day_of_year )
    index_date_dayb4 = np.logical_and(f107_data['Date'].year==dateval.year , f107_data['Date'].dayofyear==dateval.day_of_year-1 )
    f107a = float(np.squeeze(np.asarray(f107_data['f107a_earth'])[index_date]))
    f107d_dayb4 = float(np.squeeze(np.asarray(f107_data['f107d_earth'])[index_date_dayb4]))
    
    data_DTM_in.loc[i, ['f1']]    =  f107d_dayb4 # instantaneous flux at (t - 24hr)  
    data_DTM_in.loc[i, ['fbar1']] =  f107a       # mean flux of last 81 days at t 


    ##### ---- CONSTRUCT STORMTIME AP VALUES
    #####----------------------------------------------------------------
    
    dates  = dateval

    index_date3hr = np.logical_and(f107_data['Date_3hrAp'].year==dateval.year , f107_data['Date_3hrAp'].dayofyear==dateval.day_of_year  )
    indexvals =  [i for i, x in enumerate(index_date3hr) if x]
    Ap_doy_windows = f107_data['Date_3hrAp'][indexvals]

    #### Find the Current 3hr Kp window:A
    Ap_windw_hrs = [i.hour for i in Ap_doy_windows]
    Ap_windw_hrs = np.append(np.array(Ap_windw_hrs),24)  ## add the final window edge

    index_current_Ap = int(np.digitize([dates.hour],Ap_windw_hrs))
    if index_current_Ap==8:
        index_current_Ap += -1
    indexglobal_currentAp = indexvals[index_current_Ap]

#     Ap_3HR_current        = f107_data['Ap'][indexglobal_currentAp]
    Kp_3HR_prior          = f107_data['Kp'][indexglobal_currentAp-1]
    Kp_24hr_priorAVG = np.mean(f107_data['Kp'][indexglobal_currentAp-8 :indexglobal_currentAp ] ) ### 24 to 0 hours

    data_DTM_in.loc[i, ['kp1']]   = Kp_3HR_prior # kp delayed by 3 hours
    data_DTM_in.loc[i, ['kp2']]   = Kp_24hr_priorAVG  # mean of last 24 hours



In [16]:
data_DTM_in

,DOY,f1,f2,fbar1,fbar2,kp1,kp2,kp3,kp4,alt_km,localtime_rads,lat_rads,lon_rads
0,3.0,132.048251,0.0,107.058864,0.0,3.7,3.2250,0.0,0.0,225.746827,3.977452,-0.247987,-0.818713
1,3.0,132.048251,0.0,107.058864,0.0,3.7,3.2250,0.0,0.0,229.439522,4.021479,-0.303787,-0.777304
2,3.0,132.048251,0.0,107.058864,0.0,3.7,3.2250,0.0,0.0,233.444168,4.067060,-0.358886,-0.734341
3,3.0,132.048251,0.0,107.058864,0.0,3.7,3.2250,0.0,0.0,237.729079,4.114552,-0.413132,-0.689467
4,3.0,132.048251,0.0,107.058864,0.0,3.7,3.2250,0.0,0.0,242.267227,4.164340,-0.466353,-0.642297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5372,3.0,127.143177,0.0,106.914689,0.0,0.0,1.4625,0.0,0.0,285.681235,1.542037,0.923513,1.659847
5373,3.0,127.143177,0.0,106.914689,0.0,0.0,1.4625,0.0,0.0,282.872266,1.655668,0.930459,1.770860
5374,3.0,127.143177,0.0,106.914689,0.0,0.0,1.4625,0.0,0.0,279.980503,1.770554,0.930933,1.883128
5375,3.0,127.143177,0.0,106.914689,0.0,0.0,1.4625,0.0,0.0,277.019639,1.884688,0.924901,1.994644


## Save DTM2020 inputs to CSV

In [ ]:
path_data_DTM_in = "/data/zach_work/side_projects/ManualModelRuns/data_DTM_in.csv"

# DOY   f1   f2   fbar1   fbar2   kp1   kp2   kp3   kp4   alt_km   localtime_rads   lat_rads   lon_rads
space3 = "   "
DOY            =
f1             =
f2             =
fbar1          =
fbar2          =
kp1            =
kp2            =
kp3            =
kp4            =
alt_km         =
localtime_rads =
lat_rads       =
lon_rads       =
row = ""

file.write(f"{datetime.strftime(datetime.fromtimestamp(results['utc_time'][ii]), '%y%m%d%H%M%S')}  {results['c1'][ii]:9.4f}  {results['c2'][ii]:9.4f}  {results['c3'][ii]:9.4f}  {valrho:15.6e}  {nden_O:12.5e}  {nden_O2:12.5e}  {nden_He:12.5e}  {nden_N2:12.5e}   {results[temp_var][ii]:8.4e}\n")
    ###!!!!!   NOTE THERE IS AN EXTRA SPACE B4 TEMPERATURE BECAUSE FORTRAN SUCKS

# data_DTM_in.to_csv(path_data_DTM_in,
#                   sep=',',
#                   columns=data_DTM_in.columns, 
#                   header=False)

In [17]:
#                   ! ** INPUTS **
#                   !     day   =   day of year [1-366]
#                   !     f     =   f(1) = instantaneous flux at (t - 24hr)    
#                   !               f(2) = 0.
#                   !     fbar  =   fbar(1) = mean flux of last 81 days at t  
#                   !               fbar(2) = 0.
#                   !     akp   =   akp(1) = kp delayed by 3 hours, 
#                   !               akp(3) = mean of last 24 hours,
#                   !               akp(2) = 0. 
#                   !               akp(4) = 0.
#                   !     alti  =   altitude (in km) greater than 120 km
#                   !     hl    =   local time (in radian: 0-24hr = 0-2pi)
#                   !     alat  =   latitude (in radian)
#                   !     xlon  =   longitude (in radian)
#                   !
#                   ! ** OUTPUT **
#                   !     tz     =  temperature at altitude -> alti
#                   !     tinf   =  exospheric temperature
#                   !     d(1)   =  partial density of atomic hydrogen (in gram/cm3)
#                   !     d(2)   =  partial density of helium
#                   !     d(3)   =  partial density of atomic oxygen
#                   !     d(4)   =  partial density of molecular nitrogen
#                   !     d(5)   =  partial density of molecular oxygen
#                   !     d(6)   =  partial density of atomic nitrogen
#                   !     ro     =  total density (in gram/cm3)
#                   !     wmm    =  mean molecular mass (in gram)


